# Genetic Algorithm Optimization of Neural Network Architecture

A Neural Network in a powerful classification/regression tool and one of the main algorithms that has got so many people excited about Machine Learning as of late. It takes inspiration from the hierarchical structure of the visual processing systems found in many animals including humans. At it's core, it is a general function approximator. It can be trained to model any input-output relationship for which example data is available.

It consists of layers of nodes, each node in each layer connected to each node in the previous layer with a weighted edge. The first layer is called the input layer, the last layer is called the output layer and any layers in between are called hidden layers.

It is mathematically quite simple. Each node calculate a weighted sum of the inputs and applies an activation function before passing it to nodes in the next layer. This activation function is used to add non-linearity to the system so non-linear functions can be modelled.

What tends to happen with these networks once they are trained, especially when deeper architectures are used is that the first layers find low level features from the inputs. In the case of image processing, the neurons in the first layers activate(have high values) for different types of lines and curves in the image while the middle layers activate for different combinations of the lower level features such as basic shapes. The final layers activate for even higher level features such as (if we're tring to classify faces) ears, eyes and noses. And of course the neurons in the final layer activate for the highest level features we have which are different faces. These are exactly the kind of hierarchical representations which are built up in biological visual processing systems.

![](nn1.jpeg)


## The problem
Despite the astounding amount of AI research in recent years, there are still a lot of things we don't understand about how neural networks work. For example, there is no closed form solution to the optimal architecture of a neural network given a learning task, so heuristic methods are used. Most Neural Network architectures are chosen based on the engineer's past experience of which architectures work well on similar problems. Methods for automatic optimization of architecture are computationally expensive, requiring lots of trials, and generally utilize genetic algorithms or reinforcement learning. Due to the compute-intensive nature of these methods, they are only used for critical applications and problems which require small networks so they are less computationally expensive to train.

## Genetic Algorithms
In this tutorial, I will be teaching you how to implement a genetic algorithm which can perform a search over the architectures and return the most promising ones. So, what is a genetic algorithm? It is a powerful optimization technique inpired by the proccess of evolution which can be applied to any problem at all.

The basic idea is that you have a <u>population</u> of candidate solutions, all of which are tested to get a <u>fitness</u> score representing how good the solution is. The solutions with the lowest fitnesses are removed from the population and replaced by other solutions which are produced by <u>breeding</u>/combining the candidates with the highest fitnesses. All of the candidates then have small <u>mutations</u> applied. Each time a breeding+mutation operation is performed, one <u>generation</u> is said to have passed. Over a long enough period of generations, only the solutions with the best scores will survive this <u>selection proccess</u> and hence you have heuristically optimal solutions.

![](ga1.png)

## Implementation
To really understand how this method works, we will implement it to find an optimal architecture of a fully connected network for the MNIST image classification dataset. We will be using the popular deep learning library PyTorch to implement the neural networks.

The first step is to find a way to encode each possible solution into a 'gene'. In this case, we need a gene which can represent any arbitrary architecture such that we can instantiate the given neural network from it. I have chosen the gene to be a list of maximum length = <b>max_depth</b>, with each integer in the list representing the number of neurons in that hidden layer.

We now need a function to instantiate the NN from the gene so we can test it. This is done in the <b>nn_constructor</b> function and is created using the PyTorch Sequential container.

Breeding in this case is done by combining the two parent genes into one long gene and randomly selecting out of the long gene to create the child's gene. Mutation is performed by simply adding a randomly generated 0-centered integer to each chromosone of each gene in the whole population. A chromosone is a single number, encoding the number of neurons in a particular layer, in a gene array.

We also define the <b>evaluate_fitness</b> function which we will overwrite later to train the candidate on the dataset and return it's fitness. Once those functions are in place, they simply need to be called in the <b>run</b> function. The population of size = <b>pop_size</b> is initialized randomly within the bounds of the <b>max_depth, min_neurons</b> and <b>max neurons</b> parameters. min_neurons and max_neurons represent the minimum and maximum number of neurons per layer.

From each gene in the population, a neural network is constructed and trained on the dataset for the specified number of epochs. The fitness is calculated as 1/cost since we want to minimize cost and maximize fitness. The population is ordered by their fitness scores before the breeding and mutation functions are called. This loop repeats for the specified number of generation <b>gens</b>, with the genes and fitness scores of the top 5 candidates printed at each generation.

The fitness of an architecture is not the test accuracy. So to calculate this, we can call <b>test_best</b> which will instantiate a network with the best architecture of the current generation, train it for a few epochs and calculate it's test accuracy.

Note: Due to the way the mutation and crossover/breeding operations have been define in this particular implementation, some proposed architectures may exceed the given size constraints.

In [1]:
import random
from operator import itemgetter, add

import numpy as np

import torch
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'


#This class can be used to optimize the structure of a fully connected network for any problem. 
#Gene takes form of [hiddenlayer1_width, hiddenlayer2_width, hiddenlayer3_width ...]
class NNGA():
    def __init__(self, input_size, output_size, max_depth=5, min_neurons=30, max_neurons=500):
        self.max_depth = max_depth
        self.max_neurons = max_neurons
        self.min_neurons = min_neurons
        self.input_size = input_size
        self.output_size = output_size
    
    
    def nn_constructor(self, gene):        
        '''Takes in the gene as a parameter and returns the neural network the gene codes for'''
        #sequence will contain each layer of the neural network
        sequence = []
        #add input layer
        sequence.append(torch.nn.Linear(self.input_size, gene[0]))
        sequence.append(torch.nn.ReLU())
        
        #add hidden layers encoded in gene
        for k in range(1, len(gene)):
            sequence.append(torch.nn.Linear(gene[k-1], gene[k]))
            sequence.append(torch.nn.ReLU())
        
        #add output layer
        sequence.append(torch.nn.Linear(gene[-1], self.output_size))
        sequence.append(torch.nn.Softmax(dim=1))
        
        #Sequential function takes in a list of layers and returns a neural network model
        nn = torch.nn.Sequential(*sequence).to(device)

        return nn
    
    #crossover operation between two parent genes
    def crossover(self, gene1, gene2):
        '''Created a combined gene by splicing the two parent genes together
        and samples from it to create the child gene'''
        child = []
        maxlen = max(len(gene1), len(gene2))
        combined = []
        for k in range(maxlen):
            if k<len(gene1):
                combined.append(gene1[k])
            if k<len(gene2):
                combined.append(gene2[k])

        for chrom in combined:
            prob_sample = random.uniform(0, 1)
            if prob_sample>=0.5:
                child.append(chrom)
        if len(child)==0:
            child.append(gene1[0])
        return child
    
    def breed(self, breed_percent):
        '''Replace bottom breed_percent% of the population with children created
        by performing crossover operation on best performing candidates' genes '''
        number_replace = int(len(self.population_genes)*breed_percent)
        for k in range(number_replace):
            parents = random.sample(self.population_genes[number_replace:], 2)
            child = self.crossover(parents[0], parents[1])
            self.population_genes[k] = child

    def mutate(self, mutation_factor):
        '''Add noise to each candidate's gene in the population'''
        for k, gene in enumerate(self.population_genes):
            self.population_genes[k] =  [int(x) for x in map(add, list(map(round, np.random.randn(len(gene))*mutation_factor )), gene)]
    
    def evaluate_fitness(nn, epochs=1, lr=0.001):
        print("You need to overwrite this function with a function that takes a neural network in as a parameter and returns it's fitness")
        assert False
        
    def run(self, pop_size, gens, epochs=1, lr=0.001, n_top=5):
        '''Perform the optimization using the rest of the functions defined in this class'''
        #initialize population
        self.population_genes = []
        fitnesses = [0]*pop_size
        
        #initialize the population with random genes
        for k in range(pop_size):
            num_layers = random.randint(1, self.max_depth+1)
            self.population_genes.append(random.sample(range(self.min_neurons, self.max_neurons+1), num_layers))
        
        #for each generation
        for g in range(gens):
            #evaluate fitness of population
            for k, gene in enumerate(self.population_genes):
                nn = self.nn_constructor(gene)
                fitness = self.evaluate_fitness(nn, epochs=epochs, lr=lr)
                fitnesses[k] = fitness

            #sort population_genes by fitness
            self.population_genes, fitnesses = zip(*sorted(list(zip(self.population_genes, fitnesses)), key=itemgetter(1)))
            self.population_genes, fitnesses = list(self.population_genes), list(fitnesses)

            #print generation stats
            print('Generation ', g+1 , ' top ', n_top, ' architectures: ')
            for gene, fitness in zip(self.population_genes[-n_top:], fitnesses[-n_top:]):
                print('Gene: ', gene, ' Fitness: ', fitness)
            print()
            
            if g!=(gens-1): #no need to breed and mutate on final generation
                #breed to replace least fit members of population
                self.breed(0.7)

                #mutate all members
                self.mutate(6)
    
    def test_best(self, epochs=5, lr=0.001):
        '''Constructs the neural network from the best candidate in the gene pool
        and trains it for the given number of epochs before testing its accuracy'''
        nn = self.nn_constructor(self.population_genes[-1])
        fitness = self.evaluate_fitness(nn, epochs=epochs, lr=lr)
        accuracy = self.test_nn(nn)
        return self.population_genes[-1], accuracy
    
    def test_nn(self, nn):
        print("You need to overwrite this function with a function that takes a neural network in as a parameter and returns it's test accuracy")
        assert False

The MNIST dataset is downloaded using torchvision's datasets interface and loaded into a dataloader object which allows us to iterate over the data in batches. The dataloaders are used in the <b>evaluate_fitness</b> and <b>test_nn</b> functions to calculate the cost and accuracy of our model.

In [2]:
import torchvision
from torchvision import datasets, transforms

#downloads the required data from the torchvision datasets library
train_data = datasets.MNIST(root='mnist-data/',
                                  transform=transforms.ToTensor(),
                                  train=True,
                                  download=True,
                               )

train_data, val_data = torch.utils.data.random_split(train_data, [50000, 10000])

test_data = datasets.MNIST(root='mnist-data/',
                           transform=transforms.ToTensor(),
                           train=False,
                            )
train_size = len(train_data)
val_size = len(val_data)
test_size = len(test_data)

#set batch size
batch_size = 50

# dataloaders are generators that can sample from the training and test set
train_samples = torch.utils.data.DataLoader(dataset=train_data,
                                              batch_size=batch_size,
                                              shuffle=True)

val_samples = torch.utils.data.DataLoader(dataset=val_data,
                                              batch_size=batch_size,
                                              shuffle=True)

test_samples = torch.utils.data.DataLoader(dataset=test_data,
                                            batch_size=batch_size,
                                            shuffle=False)

print('Data loaded successfully')

Data loaded successfully


/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
def evaluate_fitness(nn, epochs=1, lr=0.001):
    optimizer = torch.optim.Adam(nn.parameters(), lr=lr, weight_decay=0.0001)
    for e in range(epochs):
        for b, (x, y) in enumerate(train_samples):
            x, y = x.to(device), y.to(device)
                
            h = nn.forward(x.view(-1, 784)) #calculate hypothesis
            cost = F.cross_entropy(h, y) #calculate cost
            
            optimizer.zero_grad() #zero gradients
            cost.backward() # calculate derivatives of cost wrt weights
            optimizer.step() #update parameters
    valcost=0
    for b, (x, y) in enumerate(val_samples):
            x, y = x.to(device), y.to(device)
            
            h = nn.forward(x.view(-1, 784)) #calculate hypothesis
            cost = F.cross_entropy(h, y, reduction='sum') #calculate cost
            valcost += cost.item()
            
            optimizer.zero_grad() #zero gradients
            cost.backward() # calculate derivatives of cost wrt weights
            optimizer.step() #update parameters
    valcost/=val_size

    return 1/valcost

def test_nn(nn):
    correct=0
    for b, (x, y) in enumerate(test_samples):
        x, y = x.to(device), y.to(device)
        h = nn.forward(x.view(-1, 784))
        pred = h.data.max(1)[1]
        correct += pred.eq(y).sum()
    
    accuracy = float(correct)/ test_size
    return accuracy


#instantiate the genetic architecture optimizer from the class defined earlier
mnistnn = NNGA(max_depth=4, min_neurons=30, max_neurons=300, input_size=784, output_size=10)

#overwrite the required functions
mnistnn.evaluate_fitness = evaluate_fitness
mnistnn.test_nn = test_nn

In [4]:
#run our optimizer
mnistnn.run(pop_size=15, gens=3, epochs=2)
print('Genetic optimization completed successfully')

Generation  1  top  5  architectures: 
Gene:  [61, 247]  Fitness:  0.644920278820757
Gene:  [61]  Fitness:  0.6452291611246515
Gene:  [228, 84]  Fitness:  0.646313119011836
Gene:  [177]  Fitness:  0.6497739582582497
Gene:  [256]  Fitness:  0.6514592583758121

Generation  2  top  5  architectures: 
Gene:  [175]  Fitness:  0.6495626639421007
Gene:  [174]  Fitness:  0.6496479161270281
Gene:  [251]  Fitness:  0.6500552936570556
Gene:  [250]  Fitness:  0.6504085054143354
Gene:  [255]  Fitness:  0.6506840623248719

Generation  3  top  5  architectures: 
Gene:  [253]  Fitness:  0.649385509795682
Gene:  [179]  Fitness:  0.649935111214751
Gene:  [176]  Fitness:  0.6505444714065847
Gene:  [251]  Fitness:  0.650575899691568
Gene:  [248]  Fitness:  0.6507744202195072

Genetic optimization completed successfully


In [55]:
#test the accuracy of the best candidate in the population
print(mnistnn.test_best())

([177], 0.9553)


As you can see, we can relatively quickly find an architecture that can get an accuracy of over 95% on the MNIST without having to spend hours watching the network train and manually tuning the architecture. We could instead leave an algorithm like this running overnight and wake up to a solved problem.

Image sources:<br>
https://genetic.io/en/introduction-genetic-algorithms/ <br>
https://medium.com/diaryofawannapreneur/deep-learning-for-computer-vision-for-the-average-person-861661d8aa61
